<a href="https://colab.research.google.com/github/dinh-thang/COS30018-Project-C/blob/main/falcon-7b%20and%201b%20comparison/falcon_1b_pubmed_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook finetune Falcon-rw-1b on the pubmed_qa dataset https://huggingface.co/datasets/pubmed_qa in order to record its performance and result.

This will then be used to compare with the perfomance and result of the Falcon-7b-sharded-bf16 model with the same dataset.

In [1]:
!pip install -q transformers einops
!pip install git+https://github.com/huggingface/accelerate
!pip install transformers[torch]
!pip install git+https://github.com/huggingface/peft.git
!pip install datasets bitsandbytes wandb trl
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.0 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-falep7vm
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-falep7vm
  Resolved https://github.com/huggingface/accelerate to commit 5d558f21e27744491219002a375e158eee987e0c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.23.0.dev0-py3-none-any.whl size=257231 sha256=2e80b965a2f889c7775fa218c092a95dde729db2a82b53ff1ee69f2

In [2]:
from datasets import load_dataset

dataset_name = "pubmed_qa"
dataset = load_dataset(dataset_name, "pqa_artificial")

In [7]:
formatted_dataset = dataset["train"].train_test_split(test_size=3000, train_size=27000)

def merge_context(record):
  record["context"] = "".join(record["context"]["contexts"])
  return record

def generate_text_field(record):
  record['text'] = "### Context: " + record['context'] + "### Question: " + record['question'] + "### Answer: " + record['long_answer']
  return record


training_dataset = formatted_dataset["train"]
training_dataset.add_column(name="text", column=training_dataset["long_answer"])

training_dataset = training_dataset.map(merge_context)
training_dataset = training_dataset.map(generate_text_field, remove_columns=formatted_dataset["train"].column_names)




Flattening the indices:   0%|          | 0/27000 [00:00<?, ? examples/s]

Map:   0%|          | 0/27000 [00:00<?, ? examples/s]

Map:   0%|          | 0/27000 [00:00<?, ? examples/s]

In [8]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "tiiuae/falcon-rw-1b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [11]:
from peft import LoraConfig

lora_alpha = 32
lora_dropout = 0.05
lora_r = 16

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

In [12]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4 # cang be => bias cang it
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [13]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=training_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:133: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/27000 [00:00<?, ? examples/s]

In [14]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [15]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,1.855100
20,1.843800
30,1.894200
40,1.988300
50,2.067300
60,1.782700
70,1.823400
80,1.897300
90,1.953900
100,2.033500


TrainOutput(global_step=500, training_loss=1.89194881439209, metrics={'train_runtime': 3641.5405, 'train_samples_per_second': 2.197, 'train_steps_per_second': 0.137, 'total_flos': 1.182425019482112e+16, 'train_loss': 1.89194881439209, 'epoch': 0.3})